In [ ]:
from qiskit import QuantumCircuit
import numpy as np

def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in the Deutsch-Jozsa algorithm.
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.compose(function, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

In [ ]:
from qiskit_aer import AerSimulator

def dj_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch-Jozsa function is constant or balanced.
    """
    qc = compile_circuit(function)

    result = AerSimulator().run(qc, shots=500, memory=True).result()
    measurements = result.get_memory()
    if "1" in measurements[0]:
        return "balanced"
    return "constant"

In [ ]:
def subject_constant_oracle():
    qc = QuantumCircuit(4)
    qc.x(3)

    return qc

f = subject_constant_oracle()
display(f.draw("mpl"))
display(dj_algorithm(f))

In [ ]:
def subject_balanced_oracle():
    qc = QuantumCircuit(4)
    qc.x([0, 1, 2])
    qc.cx(0, 3)
    qc.cx(1, 3)
    qc.cx(2, 3)
    qc.x([0, 1, 2])

    return qc

f = subject_balanced_oracle()
display(f.draw("mpl"))
display(dj_algorithm(f))

In [ ]:
# [IMPORTANT]
# This is the code provided by the subject to generate a random
# balanced circuit to test our Deutsch-Jozsa algorithm

import numpy as np
import qiskit as q
from qiskit import QuantumCircuit

nqubits = 4
oracle = QuantumCircuit(nqubits, name='oracle$_\omega$')

# Balanced oracle
b = np.random.randint(1,2**nqubits-1)
b_str = format(b, '0'+str(nqubits-1)+'b')
for qubit in range(len(b_str)):
    if b_str[qubit] == '1':
        oracle.x(qubit)

for qubit in range(nqubits-1):
    oracle.cx(qubit, nqubits-1)
for qubit in range(len(b_str)):
    if b_str[qubit] == '1':
        oracle.x(qubit)

oracle.to_gate()
display(oracle.draw("mpl"))

# Add it to your circuit with (and remove your personal oracle):
#your_circuit.append(oracle, range(nqubits))
display(dj_algorithm(oracle))

# [NOTE]
# The exercise asks to test with 4 qubits. But if the student has made a generic system, you can change nqubits.
# circuit is a QuantumCircuit

In [ ]:
# [IMPORTANT]
# This is the code provided by the subject to generate a random
# balanced circuit to test our Deutsch-Jozsa algorithmdisplay(dj_algorithm(oracle))

import numpy as np
import qiskit as q
from qiskit import QuantumCircuit

nqubits = 4
oracle = QuantumCircuit(nqubits, name='oracle$_\omega$')

# Constant oracle
if np.random.randint(2) == 1:
    oracle.x(nqubits-1)

oracle.to_gate()
display(oracle.draw("mpl"))

# Add it to your circuit with (and remove your personal oracle):
#your_circuit.append(oracle, range(nqubits))
display(dj_algorithm(oracle))

# [NOTE]
# The exercise asks to test with 4 qubits. But if the student has made a generic system, you can change nqubits.
# circuit is a QuantumCircuit